In [1]:
%matplotlib notebook

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os 
from tqdm import tqdm

import multiprocessing as mp

import time 
from natsort import natsorted
import moviepy.video.io.ImageSequenceClip


In [6]:
from simulation import *
from visualizer import renderParticles, drawTree, plotter

In [4]:
plt.ioff()

In [8]:
n = 100
# particles = np.random.random((n, 4)) * 2 - 1

particles1 = np.random.normal(0.6, 0.1, (n, 4))
particles2 = np.random.normal(-0.6, 0.1, (n, 4))

particles = np.vstack((particles1, particles2))
particles[:, 2:] = 0
particles[:, 2:] = 0

# FIG, AX = plt.subplots(1, 1, figsize=(6, 6))
# AX.set_facecolor("black")

tpoints, particles_list, trees = leapfrog(particles.copy(), t_start=0, t_end=10, N=1e3, L=2, theta=0.5)
# plt.scatter(particles[:, 0], particles[:, 1], c='white', s=1)

# plotter(AX, trees[0], 'white', alpha=1, zorder=99)


# plt.xlim(-1.9, 1.1)
# plt.ylim(-2.05, 1.05)
# plt.show()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [03:00<00:00,  5.53it/s]


In [10]:

tstart = time.time() 

print('[{:.2f}] Creating plot directory ...'.format(time.time() - tstart))
plot_dir = '{}/{}'.format(os.path.abspath(os.getcwd()), 'L{}n{}'.format(trees[0].length, n))
if not os.path.isdir(plot_dir):
    os.mkdir(os.path.join(plot_dir))

print('[{:.2f}] Initializing multiprocessing ...'.format(time.time() - tstart))
n_cpu = mp.cpu_count()
pool = mp.Pool(processes=n_cpu)

mp_trees = np.array_split(trees, n_cpu)
mp_particles = np.array_split(particles_list, n_cpu)
mp_tpoints = np.array_split(np.arange(len(tpoints)), n_cpu)
    
print('[{:.2f}] Creating plots ...'.format(time.time() - tstart))
for i in range(n_cpu):
    pool.apply_async(renderParticles, args=(mp_particles[i], mp_trees[i], mp_tpoints[i], plot_dir))

pool.close()
pool.join()
print('[{:.2f}] Multiprocess concluded ...'.format(time.time() - tstart))


print('[{:.2f}] Creating video ...'.format(time.time() - tstart))

fps=30 #number of frames per second
image_files = natsorted([os.path.join(plot_dir,img) for img in os.listdir(plot_dir) if img.endswith(".png")], reverse=False)
clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
clip.write_videofile('test.mp4')

print('[{:.2f}] Video created ...'.format(time.time() - tstart))

_ = [os.remove(image_file) for image_file in image_files]
os.rmdir(plot_dir)
print('[{:.2f}] Directory cleaned ...'.format(time.time() - tstart))


[0.00] Creating plot directory ...
[0.00] Initializing multiprocessing ...
[0.07] Creating plots ...
[98.67] Multiprocess concluded ...
[98.67] Creating video ...
Moviepy - Building video test.mp4.
Moviepy - Writing video test.mp4



Moviepy - Done !
Moviepy - video ready test.mp4
[108.44] Video created ...
[108.48] Directory cleaned ...
